# Readme

The MCPD claim that they allocate officers (and make traffic stops) based on accidents in each area. This file examines the veracity of that claim

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

In [1]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None)
import numpy as np
import itertools
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from shapely.geometry import Point
# from tqdm import tqdm

In [2]:
df = pd.read_csv(r"C:\Users\mikha\Dropbox\mikhael_misc\Projects\Policing Thesis\New Modified Dataset.csv")

# Cleaning Data

## Create "Unique" Var (but now it's loaded - no need to run)

In [4]:
# unique = []
# for i in list(range(1,len(df))):
#     if df['Stop ID'].iloc[i-1] != df['Stop ID'].iloc[i]:
#         unique.append(1)
#     else:
#         unique.append(0)
     
# unique = [1] + unique

# del df['First Unique']

# df.insert(1, 'First Unique', unique)


# df.to_csv(r"C:\Users\mikha\Dropbox\mikhael_misc\Projects\Policing Thesis\New Modified Dataset.csv", index=False)

## Define Race set

In [5]:
race_set = {'ASIAN', 'BLACK', 'HISPANIC', 'NATIVE AMERICAN', 'OTHER', 'WHITE'}
race_set_w_total = {'ASIAN', 'BLACK', 'HISPANIC', 'NATIVE AMERICAN', 'OTHER', 'TOTAL', 'WHITE'}

## Injuries without Accidents

Some observations have injuries but no accidents. I remove those

### First, confirm that they're randomly distributed

In [6]:
# gdf = gpd.GeoDataFrame(
#     df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

In [7]:
# gdf[(gdf['Accident']==0) & (gdf['Personal Injury']==1) & (gdf['Latitude']!=0) & (gdf['Longitude']<=-75)].plot(figsize=(15,15), markersize=.4)

### Now remove them

In [8]:
df = df[~((df['Accident']==0) & (df['Personal Injury']==1))]

# Data Wrangling

*coordinates_dict[ longitude_i ][ latitude_j ][ FIELD** ][ race ] = information for that area*

** "field" equals things like "Number of Charges @ 9 MPH", "Average Speed NOT in 9,14 MPH", etc

## Set up *coordinates_dict*

In [ ]:
CELL_WIDTH = 0.0001
DECIMAL_PLACES = 4

MIN_LONGITUDE = -77.6
MAX_LONGITUDE = -76.8

MIN_LATITUDE = 38.92
MAX_LATITUDE = 39.5

def rounder(x):#rounds to nearest CELL_WIDTH; X decimal places
    return np.around(CELL_WIDTH * round(x/CELL_WIDTH), DECIMAL_PLACES)

TOTAL_CELLS = round((((MIN_LONGITUDE-CELL_WIDTH)-(MAX_LONGITUDE+CELL_WIDTH))/CELL_WIDTH)*(((MIN_LATITUDE-CELL_WIDTH)-(MAX_LATITUDE+CELL_WIDTH))/CELL_WIDTH))
print('total # cells ~ ', TOTAL_CELLS)

coordinates_dict = {}
for x in np.arange(MIN_LONGITUDE-CELL_WIDTH, MAX_LONGITUDE+CELL_WIDTH, CELL_WIDTH):
    coordinates_dict[rounder(np.around(x,DECIMAL_PLACES))] = {}

for x in coordinates_dict:
    for y in np.arange(MIN_LATITUDE-CELL_WIDTH, MAX_LATITUDE+CELL_WIDTH, CELL_WIDTH):
        coordinates_dict[x][rounder(np.around(y,DECIMAL_PLACES))] = {
#                                                 "All Speeds Over Limit": {race: [] for race in race_set_w_total},
#                                                "Number of Speeding Charges":{race: 0 for race in race_set_w_total},
#                                                "Average Speed NOT in 9,14 MPH": {race: 0 for race in race_set_w_total},
                                               
#                                                  "Number of Charges @ 9 MPH": {race: 0 for race in race_set_w_total},
#                                                 "Number of Charges in 9,14 MPH": {race: 0 for race in race_set_w_total},
#                                                  "Number of Charges in 10,14 MPH": {race: 0 for race in race_set_w_total},
#                                                 "Number of Charges NOT in 9,14 MPH": {race: 0 for race in race_set_w_total}, 
                                                
#                                                  "Percent of Own Race's Charges @ 9 MPH": {race: 0 for race in race_set_w_total},
#                                                 "Percent of Own Race's Charges in 9,14 MPH": {race: 0 for race in race_set_w_total},
#                                                  "Percent of Own Race's Charges in 10,14 MPH": {race: 0 for race in race_set_w_total},
#                                                 "Percent of Own Race's Charges NOT in 9,14 MPH": {race: 0 for race in race_set_w_total}, 

#                                                 "Percent of Charges that were CHANGED": {race: 0 for race in race_set_w_total}, 
#                                                 "Percent of Charges that were NOT CHANGED": {race: 0 for race in race_set_w_total}, 
                                                                    
# #                                                  "As Percent of All Races' Charges @ 9 MPH" : {race: 0 for race in race_set_w_total},
# #                                                 "As Percent of All Races' in 9,14 MPH": {race: 0 for race in race_set_w_total},
# #                                                  "As Percent of All Races in 10,14 MPH": {race: 0 for race in race_set_w_total},
# #                                                 "As Percent of All Races in 9,14 MPH": {race: 0 for race in race_set_w_total},                                                                      
                                                
#                                                 "Number of Citations for Each Stop": {race: [] for race in race_set_w_total},
#                                                 "Racial Composition": {race: 0 for race in race_set_w_total},
                                                "Total_Observations": 0,
                                                "Total_Citations": 0,
                                                "Total_Accidents": 0,
                                                "Total_Injuries": 0,
                                                "Total_Fatalities": 0,
                                                "Total_Stops": 0,
            
                                                "Total_ASIAN_Stops": 0,
                                                "Total_BLACK_Stops": 0,            
                                                "Total_HISPANIC_Stops": 0,                                                                    
                                                "Total_OTHER_Stops": 0,
                                                "Total_WHITE_Stops": 0,
            
                                                "Total_Speeds_Altered": 0,
                                                "Total_Speeds_Not_Altered": 0
        
        }

        
        


total # cells ~  46427604


### Def Rounding Function

In [3]:
def rounder(x):
    """
    Rounds to nearest CELL_WIDTH; decimal places
    """
    
    return np.around(CELL_WIDTH * round(x/CELL_WIDTH), DECIMAL_PLACES)

In [ ]:
def rounder2(x):
    """
    Rounds to nearest CELL_WIDTH; decimal places
    """
    round(x, DECIMAL_PLACES)
    
    
#     return np.around(CELL_WIDTH * round(x/CELL_WIDTH), DECIMAL_PLACES)

## Fill *coordinates_dict*

In [12]:
# zipped_vars=list(zip(df['Longitude'], df['Latitude'], df['Race'], df['Speed Over Posted Limit'], df['Number of Citations']))

for i,x in enumerate(list(zip(df['Longitude'], df['Latitude'], df['Race'], df['Speed Over Posted Limit'], df['Citation - (D_Violation Type)'], df['Accident'], df['Personal Injury'], df['Fatal'], df['First Unique'], df['Speed Altered']))):
        if MIN_LONGITUDE <= x[0] <= MAX_LONGITUDE: #-77.6 <= x <= -76.8
            if MIN_LATITUDE <= x[1] <= MAX_LATITUDE:

#                 coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Racial Composition" ][ x[2] ] += 1
#                 coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Racial Composition" ][ 'TOTAL' ] += 1
                
                coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_Observations" ] += 1
                coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_Citations" ] += x[4] 
                coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_Accidents" ] += x[5] # ==1 if accident
                coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_Injuries" ] += x[6]
                coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_Fatalities" ] += x[7]
                coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_Stops" ] += x[8]
                if x[2] == 'ASIAN':
                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_ASIAN_Stops" ] += x[8]
                elif x[2] == 'BLACK':
                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_BLACK_Stops" ] += x[8]
                if x[2] == 'HISPANIC':
                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_HISPANIC_Stops" ] += x[8]
                if x[2] == 'OTHER':
                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_OTHER_Stops" ] += x[8]
                if x[2] == 'WHITE':
                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_WHITE_Stops" ] += x[8]
                    
                if x[9]==1: #if speed altered (and not nan)
                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_Speeds_Altered" ] += 1
                elif x[9]==0: #if speed NOT altered (and not nan)
                    coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Total_Speeds_Not_Altered" ] += 1
                
                

                
#                 if not np.isnan(x[3]): # if df["Speed Over Posted Limit"].iloc[i] != np.nan
                    
#                     coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Speeding Charges" ][ x[2] ] += 1
#                     coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Speeding Charges" ][ 'TOTAL' ] += 1

#                     coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "All Speeds Over Limit" ][ x[2] ].append(x[3])
#                     coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "All Speeds Over Limit" ]['TOTAL'].append(x[3])

#                     coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Speeding Charges" ][ x[2] ] += 1
#                     coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Speeding Charges" ][ 'TOTAL' ] +=1

#                     coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Citations for Each Stop" ][ x[2] ].append(x[4])
#                     coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Citations for Each Stop" ][ 'TOTAL' ].append(x[4])


#                     if x[3] == 9:
#                         coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges @ 9 MPH" ][ x[2] ] += 1
#                         coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges @ 9 MPH" ][ 'TOTAL' ] += 1

#                     if 9 <= x[3] <= 14:
#                         coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges in 9,14 MPH" ][ x[2] ] += 1
#                         coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges in 9,14 MPH" ][ 'TOTAL' ] += 1

#                     if 10 <= x[3] <= 14:
#                         coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges in 10,14 MPH" ][ x[2] ] += 1
#                         coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges in 10,14 MPH" ][ 'TOTAL' ] += 1
                        
#                     if 9 > x[3] or x[3] > 14:
#                         coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges NOT in 9,14 MPH" ][ x[2] ] += 1
#                         coordinates_dict[ rounder(x[0]) ][ rounder(x[1]) ][ "Number of Charges NOT in 9,14 MPH" ][ 'TOTAL' ] += 1

## Create pd dataframe from coordinates_dict

In [13]:
#convert dict to pd df
allocation_balance_df = pd.DataFrame.from_dict({(i,j): coordinates_dict[i][j] 
                           for i in coordinates_dict.keys() 
                           for j in coordinates_dict[i].keys()},
                       orient='index')

#give names to multi-index
allocation_balance_df.index.names = ['long', 'lat']

#convert indices to cols
allocation_balance_df.reset_index(level=allocation_balance_df.index.names, inplace=True)

In [14]:
#convert to geopandas dataframe
gdf = gpd.GeoDataFrame(
    allocation_balance_df, geometry=gpd.points_from_xy(allocation_balance_df['long'], allocation_balance_df['lat']))

In [15]:
# create fields
gdf['Observations_per_Accident'] = gdf['Total_Observations'] / gdf['Total_Accidents']
gdf['Accidents_per_Observation'] = gdf['Total_Accidents'] / gdf['Total_Observations']

gdf['Observations per Injury'] = gdf['Total_Observations'] / gdf['Total_Injuries']
gdf['Injuries_per_Observation'] = gdf['Total_Injuries'] / gdf['Total_Observations']

gdf['Observations per Fatality'] = gdf['Total_Observations'] / gdf['Total_Fatalities']
gdf['Fatalities_per_Observation'] = gdf['Total_Fatalities'] / gdf['Total_Observations']

gdf['Stops_per_Accident'] = gdf['Total_Stops'] / gdf['Total_Accidents']
gdf['Accidents_per_Stop'] = gdf['Total_Accidents'] / gdf['Total_Stops']

gdf['Stops_per_Injury'] = gdf['Total_Stops'] / gdf['Total_Injuries']
gdf['Injuries_per_Stop'] = gdf['Total_Injuries'] / gdf['Total_Stops']

gdf['Citations_per_Stop'] = gdf['Total_Citations'] / gdf['Total_Stops']

gdf['pct_Stops_ASIAN'] = gdf['Total_ASIAN_Stops'] / gdf['Total_Stops']
gdf['pct_Stops_BLACK'] = gdf['Total_BLACK_Stops'] / gdf['Total_Stops']
gdf['pct_Stops_HISPANIC'] = gdf['Total_HISPANIC_Stops'] / gdf['Total_Stops']
gdf['pct_Stops_OTHER'] = gdf['Total_OTHER_Stops'] / gdf['Total_Stops']
gdf['pct_Stops_WHITE'] = gdf['Total_WHITE_Stops'] / gdf['Total_Stops']

gdf['pct_Altered'] = gdf['Total_Speeds_Altered'] / ( gdf['Total_Speeds_Altered'] + gdf['Total_Speeds_Not_Altered'] )


#replace inf with 0
gdf.replace(np.inf, 0, inplace=True)

#drop cells with less than 30 Observations
gdf = gdf[gdf['Total_Observations']>=30]

# replace nan with zero
gdf.fillna(0, inplace=True)

# normalize fields
for field in list(set(gdf.columns) - set(['long', 'lat', 'geometry'])): #all cols excep long, lat, geometry
    gdf['Normalized ' + field] = (gdf[field]-gdf[field].mean())/gdf[field].std()
    
# replace nan with zero again after normalizing
gdf.fillna(0, inplace=True)

## Distance to Nearest Police Station

In [16]:
station_locations = {} #each number corresponds to its district

# station_locations[1] = Point([39.058586719462355, -77.04866263431445])
# station_locations[2] = Point([38.99134343887121, -77.0978411276706])
# station_locations[3] = Point([39.04628489091019, -76.99114687774473])
# station_locations[4] = Point([39.05934890838263, -77.04882510089048])
# station_locations[5] = Point([39.18443844480189, -77.26277546702069])
# station_locations[6] = Point([39.15007606517369, -77.23548162468924])

station_locations[1] = Point([-77.04866263431445, 39.058586719462355])
station_locations[2] = Point([-77.0978411276706, 38.99134343887121])
station_locations[3] = Point([-76.99114687774473, 39.04628489091019])
station_locations[4] = Point([-77.04882510089048, 39.05934890838263])
station_locations[5] = Point([-77.26277546702069, 39.18443844480189])
station_locations[6] = Point([-77.23548162468924, 39.15007606517369])


for district in station_locations:
#     gdf[f'District_{district}_Station_Location'] = station_locations[district]
    
    gdf[f'Distance_to_District_{district}_Station'] = gdf.distance(station_locations[district])
    

distance_fields = [f'Distance_to_District_{district}_Station' for district in station_locations]

gdf['Distance_to_Closest_Station'] = gdf[distance_fields].min(axis=1)

In [18]:
distance_fields = [f'Distance_to_District_{district}_Station' for district in station_locations]

In [19]:
gdf['Distance_to_Closest_Station'] = gdf[distance_fields].min(axis=1)

## Export to CSV

In [22]:
geo_df = pd.DataFrame(gdf)

In [23]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

gdf.to_csv(f"Police Stops vs Accidents - width={str(CELL_WIDTH)} - {today}.csv", index=False)